In [1]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GroupKFold

## Custom libraries
import index_helpers as ih

In [2]:
## Import, index, and split
df = ih.read_and_merge_segmented_data()
df = ih.index_df_by_person(df)
df = ih.categorical_float_to_int(df)
df = ih.categorical_to_dummy(df)
X_train, X_val, y_train, y_val = ih.train_test_split_on_index(features = df.drop("Label", axis=1),
                                                             label = df["Label"])

## Modify data for GroupKFold
groups = y_train.reset_index()['File_Name_split']
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

In [5]:
### Logistic regression
from sklearn.linear_model import LogisticRegression

## Make pipeline - name classifier "clf"
clf_pipeline = Pipeline([("st_scaler", StandardScaler()),
                        ("clf", LogisticRegression())])

## Use "clf__" in order to correctly assign parameters to the clf object
clf_param_grid = {'clf__penalty':['l1', 'l2', 'elasticnet', 'none'],
                  'clf__tol':[0.0003, 0.0002, 0.0001, 0.00005], 
                  'clf__C':[2.0, 1.5, 1.0, 0.5], 
                  'clf__intercept_scaling':[1], 
                  'clf__max_iter':[50],
                 'clf__solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

## Instantiate GroupKFold to avoid data leakage - to be passed to cv
gkf=GroupKFold(n_splits=10)

## Set up Randomized search CV
clf_rand_auc = RandomizedSearchCV(estimator=clf_pipeline,
                                  param_distributions=clf_param_grid,
                                  cv=gkf, scoring='roc_auc', verbose=1, n_jobs=2, n_iter=50)

## Perform Group K-Cross-validation
clf_rand_auc.fit(X_train, y_train, groups=groups)

## Print results
print("Best score: ",  clf_rand_auc.best_score_)
print("Best estimator: ", clf_rand_auc.best_estimator_)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   30.1s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:  3.7min
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:  5.1min
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:  5.9min finished


Best score:  0.5595213702921746
Best estimator:  Pipeline(steps=[('st_scaler', StandardScaler()),
                ('clf',
                 LogisticRegression(C=0.5, max_iter=50, penalty='l1',
                                    solver='saga', tol=5e-05))])


C:\Users\chris\anaconda\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
